In [3]:
import numpy as np
import pandas as pd
from joblib import delayed, Parallel

# Run This Ensemble When Submission Available
# (Try Squaring Results and Renormalizing Next)

In [4]:
class paths:
    TRAIN = './data/train.csv'
    TEST = './data/test.csv'

In [5]:
df = pd.read_csv(paths.TRAIN)
df.describe()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
count,19219.000000,19219.000000,19219.000000,1.921900e+04,1.921900e+04,19219.000000,19219.000000,19219.000000,1.921900e+04,19219.000000,...,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000
mean,9609.000000,709.854675,753.857641,1.849756e+06,1.846605e+06,1683.987616,95.654665,64.124096,1.918467e+05,84.808419,...,0.102742,-0.138382,0.571902,0.076279,0.059837,0.178573,0.029554,0.025235,0.247828,0.341225
std,5548.191747,531.544189,499.836603,1.903554e+06,1.896295e+06,3730.319865,177.821382,101.054178,4.420247e+05,28.800344,...,0.487681,0.120344,0.332219,0.265450,0.237190,0.383005,0.169358,0.156844,0.431762,0.474133
min,0.000000,0.000000,4.000000,6.712000e+03,6.724000e+03,6.000000,2.000000,1.000000,2.500000e+02,0.000000,...,-0.988400,-0.885000,0.119000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4804.500000,49.000000,214.000000,6.574680e+05,6.575020e+05,89.000000,15.000000,14.000000,9.848000e+03,70.000000,...,-0.272700,-0.192500,0.253200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9609.000000,777.000000,796.000000,1.398169e+06,1.398179e+06,168.000000,25.000000,23.000000,1.823800e+04,90.000000,...,0.111100,-0.142600,0.472900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,14413.500000,1152.000000,1165.000000,2.368032e+06,2.362511e+06,653.000000,64.000000,61.000000,6.797800e+04,105.000000,...,0.529400,-0.084000,0.999400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,19218.000000,1705.000000,1713.000000,1.298766e+07,1.298769e+07,152655.000000,7553.000000,903.000000,1.159141e+07,196.000000,...,0.991700,0.642100,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Feature Engineering

In [6]:
def feature_engineering(df):
    new_df = df.copy()
    new_df['Sine'] = np.sqrt(1 - new_df['Orientation_Index']**2)
    new_df['Fixed_Area'] = new_df['Pixels_Areas'] / new_df['Sine']
    new_df['Fixed_LumSum'] = new_df['Sum_of_Luminosity'] / new_df['Sine']
    new_df['Norm_SumLum'] = new_df['Sum_of_Luminosity'] / new_df['Fixed_Area']
    new_df['Norm_MinLum'] = new_df['Minimum_of_Luminosity'] / new_df['Sine']
    new_df['Norm_MaxLum'] = new_df['Maximum_of_Luminosity'] / new_df['Sine']
    new_df['X_Size'] = new_df['X_Maximum'] - new_df['X_Minimum']
    new_df['X_Size_abs'] = np.abs(new_df['X_Maximum'] - new_df['X_Minimum'])

    drop_cols = ['Y_Maximum','Y_Minimum','X_Maximum','X_Minimum']
    #            'Luminosity_Index','Edges_X_Index',
    #            'Edges_Index', 'Maximum_of_Luminosity', 'Sum_of_Luminosity', 'Norm_MaxLum', 
    #            'Log_Y_Index', 'Square_Index', 'Empty_Index', 'SigmoidOfAreas']
    
    new_df = new_df.drop(drop_cols, axis=1)
    return new_df

In [7]:
train_df = feature_engineering(df)
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,19219.0,9609.000000,5548.191747,0.000000,4804.500000,9609.000000,14413.500000,1.921800e+04
Pixels_Areas,19219.0,1683.987616,3730.319865,6.000000,89.000000,168.000000,653.000000,1.526550e+05
X_Perimeter,19219.0,95.654665,177.821382,2.000000,15.000000,25.000000,64.000000,7.553000e+03
Y_Perimeter,19219.0,64.124096,101.054178,1.000000,14.000000,23.000000,61.000000,9.030000e+02
Sum_of_Luminosity,19219.0,191846.678235,442024.694057,250.000000,9848.000000,18238.000000,67978.000000,1.159141e+07
Minimum_of_Luminosity,19219.0,84.808419,28.800344,0.000000,70.000000,90.000000,105.000000,1.960000e+02
Maximum_of_Luminosity,19219.0,128.647380,14.196976,39.000000,124.000000,127.000000,135.000000,2.530000e+02
Length_of_Conveyer,19219.0,1459.350747,145.568687,1227.000000,1358.000000,1364.000000,1652.000000,1.794000e+03
TypeOfSteel_A300,19219.0,0.402674,0.490449,0.000000,0.000000,0.000000,1.000000,1.000000e+00
TypeOfSteel_A400,19219.0,0.596337,0.490644,0.000000,0.000000,1.000000,1.000000,1.000000e+00


In [8]:
train_df_A300 = train_df.loc[train_df['TypeOfSteel_A300']==1].drop(['TypeOfSteel_A300', 'TypeOfSteel_A400'], axis=1).reset_index(drop=True)
train_df_A400 = train_df.loc[train_df['TypeOfSteel_A400']==1].drop(['TypeOfSteel_A300', 'TypeOfSteel_A400'], axis=1).reset_index(drop=True)

## Splitting

In [9]:
y_cols = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']

y_cl = train_df[y_cols]
X_cl = train_df.drop(y_cl, axis=1)
ids_cl = X_cl['id']
X_cl = X_cl.drop(['id'],axis=1)

y_3 = train_df_A300[y_cols]
X_3 = train_df_A300.drop(y_cols, axis=1)
ids_3 = X_3['id']
X_3 = X_3.drop(['id'],axis=1)

y_4 = train_df_A400[y_cols]
X_4 = train_df_A400.drop(y_cols, axis=1)
ids_4 = X_4['id']
X_4 = X_4.drop(['id'],axis=1)

## Group KFold

In [10]:
from sklearn.model_selection import GroupKFold
gkf_cl = GroupKFold(n_splits=5)
gkf_3 = GroupKFold(n_splits=5)
gkf_4 = GroupKFold(n_splits=5)

In [11]:
trains_cl = []
valids_cl = []
for fold, (train_index, valid_index) in enumerate(gkf_cl.split(X_cl, y_cl, ids_cl)):
    print(f'Fold: {fold}')
    train_X = X_cl.loc[train_index]
    train_y = y_cl.loc[train_index]
    valid_X = X_cl.loc[valid_index]
    valid_y = y_cl.loc[valid_index]
    trains_cl.append((train_X, train_y))
    valids_cl.append((valid_X, valid_y))

trains_3 = []
valids_3 = []
for fold, (train_index, valid_index) in enumerate(gkf_3.split(X_3, y_3, ids_3)):
    print(f'Fold: {fold}')
    train_X = X_3.loc[train_index]
    train_y = y_3.loc[train_index]
    valid_X = X_3.loc[valid_index]
    valid_y = y_3.loc[valid_index]
    trains_3.append((train_X, train_y))
    valids_3.append((valid_X, valid_y))
    
trains_4 = []
valids_4 = []
for fold, (train_index, valid_index) in enumerate(gkf_4.split(X_4, y_4, ids_4)):
    print(f'Fold: {fold}')
    train_X = X_4.loc[train_index]
    train_y = y_4.loc[train_index]
    valid_X = X_4.loc[valid_index]
    valid_y = y_4.loc[valid_index]
    trains_4.append((train_X, train_y))
    valids_4.append((valid_X, valid_y))

Fold: 0
Fold: 1
Fold: 2
Fold: 3
Fold: 4
Fold: 0
Fold: 1
Fold: 2
Fold: 3
Fold: 4
Fold: 0
Fold: 1
Fold: 2
Fold: 3
Fold: 4


## XGBoost

In [12]:
from xgboost import XGBClassifier

In [17]:
estimators = 1000
stopping = 10
lr = 0.05

def fitter(i: int):
    xgboosts = []
    if i == 0:
        for fold in range(5):
            model = XGBClassifier(n_estimators=estimators,
                                  early_stopping_rounds=stopping,
                                  learning_rate=lr)
            model.fit(trains_cl[fold][0], trains_cl[fold][1], 
                      eval_set = [valids_cl[fold]], verbose=0)
            xgboosts.append(model)

    if i == 1:
        for fold in range(5):
            print(f'Fold: {fold}')
            model = XGBClassifier(n_estimators=estimators,
                                early_stopping_rounds=stopping,
                                learning_rate=lr)
            model.fit(trains_3[fold][0], trains_3[fold][1], 
                    eval_set = [valids_3[fold]], verbose=0)
            xgboosts.append(model)

    if i == 2:
        for fold in range(5):
            print(f'Fold: {fold}')
            model = XGBClassifier(n_estimators=estimators,
                                early_stopping_rounds=stopping,
                                learning_rate=lr)
            model.fit(trains_4[fold][0], trains_4[fold][1], 
                    eval_set = [valids_4[fold]], verbose=0)
            xgboosts.append(model)
    return xgboosts

In [18]:
xgboosts_cl, xgboosts_3, xgboosts_4 = Parallel(n_jobs=3)(delayed(fitter)(i) for i in range(3))

## Feature Importances

In [19]:
print('Classic')
print('='*50)
importances = xgboosts_cl[0].feature_importances_
for xgboost in xgboosts_cl[1:]:
    importances += xgboost.feature_importances_
importances = np.array(importances) / 5
Xcols = np.array(X_cl.columns)[np.argsort(importances)][::-1]
sortd = np.sort(importances)[::-1]
print('')
for i, col in enumerate(Xcols):
    print(f'{col}: {round(sortd[i], 2)}')
    
print(' ')

print('A300')
print('='*50)
importances = xgboosts_3[0].feature_importances_
for xgboost in xgboosts_3[1:]:
    importances += xgboost.feature_importances_
importances = np.array(importances) / 5
Xcols = np.array(X_3.columns)[np.argsort(importances)][::-1]
sortd = np.sort(importances)[::-1]
print('')
for i, col in enumerate(Xcols):
    print(f'{col}: {round(sortd[i], 2)}')
    
print(' ')

print('A400')
print('='*50)
importances = xgboosts_4[0].feature_importances_
for xgboost in xgboosts_4[1:]:
    importances += xgboost.feature_importances_
importances = np.array(importances) / 5
Xcols = np.array(X_4.columns)[np.argsort(importances)][::-1]
sortd = np.sort(importances)[::-1]
print('')
for i, col in enumerate(Xcols):
    print(f'{col}: {round(sortd[i], 2)}')

Classic

X_Size_abs: 0.28999999165534973
LogOfAreas: 0.10999999940395355
Log_X_Index: 0.05000000074505806
TypeOfSteel_A300: 0.05000000074505806
Outside_X_Index: 0.05000000074505806
Pixels_Areas: 0.03999999910593033
X_Size: 0.03999999910593033
Steel_Plate_Thickness: 0.03999999910593033
Orientation_Index: 0.029999999329447746
Fixed_Area: 0.029999999329447746
Length_of_Conveyer: 0.029999999329447746
TypeOfSteel_A400: 0.019999999552965164
Sine: 0.019999999552965164
Edges_Y_Index: 0.019999999552965164
X_Perimeter: 0.019999999552965164
Norm_MinLum: 0.019999999552965164
Outside_Global_Index: 0.009999999776482582
Minimum_of_Luminosity: 0.009999999776482582
Y_Perimeter: 0.009999999776482582
Edges_Index: 0.009999999776482582
Sum_of_Luminosity: 0.009999999776482582
Norm_MaxLum: 0.009999999776482582
Maximum_of_Luminosity: 0.009999999776482582
Log_Y_Index: 0.009999999776482582
Fixed_LumSum: 0.009999999776482582
Square_Index: 0.009999999776482582
Luminosity_Index: 0.009999999776482582
Empty_Index: 0

# Classifying

In [20]:
test = pd.read_csv(paths.TEST)
test_df = feature_engineering(test)

In [21]:
test_df_A300 = test_df.loc[test_df['TypeOfSteel_A300']==1].drop(['TypeOfSteel_A300', 'TypeOfSteel_A400'], axis=1).reset_index(drop=True)
test_df_A400 = test_df.loc[test_df['TypeOfSteel_A400']==1].drop(['TypeOfSteel_A300', 'TypeOfSteel_A400'], axis=1).reset_index(drop=True)

In [22]:
ids_test_cl = test_df['id']
X_test_cl = test_df.drop(['id'],axis=1)

ids_test_3 = test_df_A300['id']
X_test_3 = test_df_A300.drop(['id'],axis=1)

ids_test_4 = test_df_A400['id']
X_test_4 = test_df_A400.drop(['id'],axis=1)

In [23]:
# A300
preds_classic = np.array(xgboosts_cl[0].predict_proba(X_test_cl))
for fold in range(1,5):
    preds_classic += xgboosts_cl[fold].predict_proba(X_test_cl)
preds_classic = preds_classic / preds_classic.sum(axis=1).reshape(-1,1)
result_classic = pd.DataFrame(preds_classic, columns = y_cols)
result_classic['id'] = ids_test_cl

# A300
preds_A300 = np.array(xgboosts_3[0].predict_proba(X_test_3))
for fold in range(1,5):
    preds_A300 += xgboosts_3[fold].predict_proba(X_test_3)
preds_A300 = preds_A300 / preds_A300.sum(axis=1).reshape(-1,1)
result_A300 = pd.DataFrame(preds_A300, columns = y_cols)
result_A300['id'] = ids_test_3

# A400
preds_A400 = np.array(xgboosts_4[0].predict_proba(X_test_4))
for fold in range(1,5):
    preds_A400 += xgboosts_4[fold].predict_proba(X_test_4)
preds_A400 = preds_A400 / preds_A400.sum(axis=1).reshape(-1,1)
result_A400 = pd.DataFrame(preds_A400, columns = y_cols)
result_A400['id'] = ids_test_4

result_df = pd.concat([result_classic, result_A300, result_A400])
result_df = result_df.groupby('id', as_index=False).agg('mean')
display(result_df)

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.493833,0.002158,0.002752,0.001003,0.011071,0.153135,0.336048
1,19220,0.244078,0.016594,0.014261,0.001139,0.190776,0.177375,0.355778
2,19221,0.002218,0.064435,0.057306,0.001585,0.007183,0.340473,0.526801
3,19222,0.172182,0.002179,0.001675,0.002680,0.008585,0.406280,0.406420
4,19223,0.004290,0.002552,0.001765,0.002142,0.007863,0.620225,0.361164
...,...,...,...,...,...,...,...,...
12809,32028,0.087867,0.110921,0.003734,0.001307,0.035062,0.286698,0.474411
12810,32029,0.174633,0.005516,0.029003,0.007783,0.193803,0.153055,0.436207
12811,32030,0.001265,0.001355,0.934514,0.001097,0.001231,0.001413,0.059126
12812,32031,0.366505,0.011610,0.024353,0.001218,0.047436,0.181600,0.367278
